In [1]:
# display_dft in this cell creates an interactive plot of the DFT of an input waveform

import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display


def generate_time(n_samples):
    return np.linspace(0, 1, n_samples, endpoint=False)

def sine(f, t):   return np.sin(2*np.pi*f*t)
def cosine(f, t): return np.cos(2*np.pi*f*t)
def correlation_products(x, im, re):
    re_cor = re*x
    im_cor = im*x
    return re_cor, im_cor

def dft_magnitudes(x, im, re):
    """Return magnitude of current correlation signals against time signal."""
    re_cor, im_cor = correlation_products(x, im, re)
    re_coeff = np.sum(re_cor)
    im_coeff = np.sum(im_cor)
    mag = np.sqrt(re_coeff**2 + im_coeff**2)
    return mag


def display_dft(input_signal, t, freqs):
    # ---- Sampling ----
    N = len(t)
    # Initial freqs
    im, re = 0, 0
    x0 = input_signal
    re_cor, im_cor = correlation_products(x0, -sine(im, t), cosine(re, t))

    fig = go.FigureWidget(
        make_subplots(
            rows=3, cols=1,
            shared_xaxes=False,
            subplot_titles=(
                "Correlation Signal and Time Signal",
                "Point-by-point correlation products (Correlation Signals x Time Signal)",
                f"Magnitude of DFT k=[0-{len(freqs)-1}] over Time Signal"
            ),
            vertical_spacing=0.10
        )
    )

    # Correlation Signal: Imaginary part
    fig.add_scatter(
        x=t, y=-sine(im, t), mode="markers",
        marker=dict(color="red", size=9),
        name="Corr Signal Im",
        row=1, col=1
    )
    # Correlation Signal: Real part
    fig.add_scatter(
        x=t, y=cosine(re, t), mode="markers",
        marker=dict(color="green", size=9),
        name="Corr Signal Re",
        row=1, col=1
    )
    # Time Signal
    fig.add_scatter(
        x=t, y=x0, mode="markers",
        marker=dict(color="blue", size=9, symbol="circle-open"),
        name="Time Signal",
        row=1, col=1
    )
    fig.add_scatter(
        x=t, y=re_cor, mode="markers",
        marker=dict(size=10),
        name="Time Signal x Re",
        row=2, col=1
    )
    fig.add_scatter(
        x=t, y=im_cor, mode="markers",
        marker=dict(size=10),
        name="Time Signal x Im",
        row=2, col=1
    )

    # compute the magnitude of the DFT for each frequency
    mags = []
    for f in freqs:
        mag = dft_magnitudes(x0, -sine(f, t), cosine(f, t))
        mags.append(mag)
    fig.add_scatter(
        x=freqs, y=mags, mode="markers",
        marker=dict(size=10),
        name=f"Magnitude of DFT k=[0-{len(freqs)-1}]",
        row=3, col=1
    )

    # Axis labels/ranges
    fig.update_yaxes(range=[-1.2, 1.2], title_text="Amplitude", row=1, col=1)
    fig.update_xaxes(title_text="Time (s)", row=1, col=1)
    fig.update_layout(
        height=1000,
        template="plotly_white",
        title=f"discrete fourier transform with {len(t)} discrete samples"
    )
    fig.update_yaxes(title_text="Product", row=2, col=1)
    fig.update_xaxes(title_text="Time (s)", row=2, col=1)
    fig.update_yaxes(title_text="Magnitude", row=3, col=1)
    fig.update_xaxes(title_text="k", row=3, col=1)

    # ---- Independent sliders ----
    sliderA = widgets.IntSlider(
        value=0, min=0, max=len(freqs)-1, step=1,
        description="k", continuous_update=True
    )

    def update_traces(*_):
        fA = sliderA.value

        im_k = -sine(fA, t)
        re_k = cosine(fA, t)
        cor_re, cor_im = correlation_products(x0, -sine(fA, t), cosine(fA, t))

        fig.data[0].y = im_k
        fig.data[1].y = re_k

        # Update row 4 spectrum
        fig.data[3].y = cor_re
        fig.data[4].y = cor_im

    sliderA.observe(update_traces, names="value")

    display(sliderA, fig)


In [2]:
# How display_dft is used
# The slide bar allows you to select the frequency of the correlation signal
t = generate_time(30) # experiment with 30 samples
input_signal = sine(2, t) # 2 Hz sine wave as input signal
freqs = np.array([0, 1, 2, 3, 4, 5]) # Correlation signals of 0 to 5 Hz
display_dft(input_signal, t, freqs)

IntSlider(value=0, description='k', max=5)

FigureWidget({
    'data': [{'marker': {'color': 'red', 'size': 9},
              'mode': 'markers',
              'name': 'Corr Signal Im',
              'type': 'scatter',
              'uid': '8636a667-7219-47d2-8d72-bcb7ac32c656',
              'x': array([0.        , 0.03333333, 0.06666667, 0.1       , 0.13333333, 0.16666667,
                          0.2       , 0.23333333, 0.26666667, 0.3       , 0.33333333, 0.36666667,
                          0.4       , 0.43333333, 0.46666667, 0.5       , 0.53333333, 0.56666667,
                          0.6       , 0.63333333, 0.66666667, 0.7       , 0.73333333, 0.76666667,
                          0.8       , 0.83333333, 0.86666667, 0.9       , 0.93333333, 0.96666667]),
              'xaxis': 'x',
              'y': array([-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
                          -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
                          -0., -0.]),
              '